# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## 1. Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

data = pd.read_csv('/datasets/insurance.csv')
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные не содержат пропусков и все признаки числового типа, выполним масштабирование данных.

In [4]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [5]:
#Разобъем данные на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

In [6]:
scaler = StandardScaler()
scaler.fit(features_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
#Выполним масштабирование
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

Данные загружены и подготовлены.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регрессии не изменится. Параметры ЛГ в исходной задаче и в преобразованной связаны следующей формулой: $$ w' = P^{-1} w $$

**Обоснование:** $$ a' = X'w', $$ где $ X' = XP, $ где $ P $ - обратимая матрица

$$ w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} P^T X^T y, $$

т.к. $ P^T, X^T X, P $ - квадратные матрицы => можно раскрыть скобки
$$ w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y, $$
$ (P^T)^{-1} P^T = E $, где $E$ - единичная матрица, тогда
$$ w' = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w $$
$$ a' = XP P^{-1} w = Xw = a $$

## 3. Алгоритм преобразования

**Алгоритм**

Преобразуем данные, умножив матрицу признаков *features_train_scaled* обучающей выборки и *features_test_scaled* тестовой выборки на обратимую матрицу, размерами равную числу столбцов матрицы признаков.

**Обоснование**

См. шаг 2.

## 4. Проверка алгоритма

In [34]:
#Обучим линейную регрессию на исходных данных
model = LinearRegression()
model.fit(features_train_scaled, target_train)
predictions = model.predict(features_test_scaled)
r2 = r2_score(target_test, predictions)
r2

0.43522757127026657

In [39]:
#Обратимая матрица
try:
    matrix = np.random.rand(features_train_scaled.shape[1],features_train_scaled.shape[1])
    numpy.linalg.inv(matrix)
except numpy.linalg.LinAlgError:
    matrix = np.random.rand(features_train_scaled.shape[1],features_train_scaled.shape[1])
matrix

array([[0.39185222, 0.92107622, 0.1297572 , 0.72362564],
       [0.30789821, 0.36706582, 0.86082805, 0.40731051],
       [0.57169403, 0.30374427, 0.82578959, 0.32161492],
       [0.16958349, 0.57900018, 0.34162686, 0.55654463]])

In [36]:
#Преобразуем признаки с помощью матричного умножения
features_train_change = features_train_scaled @ matrix
features_test_change = features_test_scaled @ matrix

In [38]:
#Проверим модель на преобразованных данных
model_change = LinearRegression()
model_change.fit(features_train_change, target_train)
predictions_change = model_change.predict(features_test_change)
r2_change = r2_score(target_test, predictions_change)
r2_change

0.43522757127026657

Проверен алгоритм преобразования данных, метрика *R2* линейной регрессии не отличается до преобразования и после и равна 0.435.